## Imports

In [2]:
!python3 -m spacy download en
!python3 -m spacy download fr
!pip install torchinfo
!pip install einops
!pip install wandb

import math
import time
import os
import copy
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.utils.checkpoint as cp
from torch.utils.data import DataLoader
from torch.jit.annotations import List
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor, Normalize
from torchvision.transforms import Resize
from torchvision.transforms.functional_pil import resize
from torchvision.transforms import Grayscale
from torchvision.models.resnet import ResNet, BasicBlock


import numpy as np
from collections import OrderedDict
from tqdm import tqdm
from torchinfo import summary
import wandb
from PIL import Image, ImageOps, ImageEnhance

2023-05-02 03:28:46.786994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 03:28:48.262529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-02 03:28:50.595198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-02 03:28:50.595786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

**Drive**

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
torch.manual_seed(5)
torch.cuda.manual_seed_all(5)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# **Downloading and Preprocessing Data**

In [3]:
mnist_trainset = datasets.MNIST(root='./data/MNIST', train=True, download=True)
mnist_trainset.transform = transforms.Compose([Grayscale(num_output_channels=3), ToTensor(), Normalize([0.1205],[0.3015]),])
mnist_testset = datasets.MNIST(root='./data/MNIST', train=False, download=True)
mnist_testset.transform = transforms.Compose([Grayscale(num_output_channels=3), ToTensor(), Normalize([0.1307],[0.3015]),])
mnist_trainset = DataLoader(mnist_trainset, batch_size=128, shuffle=True)
mnist_testset = DataLoader(mnist_testset, batch_size=128)

CIFAR10_trainset = datasets.CIFAR10(root='./data/CIFAR', train=True, download=True)
CIFAR10_trainset.transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4),ToTensor(), Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),])
CIFAR10_testset = datasets.CIFAR10(root='./data/CIFAR', train=False, download=True)
CIFAR10_testset.transform = transforms.Compose([ToTensor(), Normalize([0.4914, 0.4822, 0.4465],[0.2023, 0.1994, 0.2010]),])
CIFAR10_trainset = DataLoader(CIFAR10_trainset, batch_size=128, shuffle=True)
CIFAR10_testset = DataLoader(CIFAR10_testset, batch_size=128)

CIFAR100_trainset = datasets.CIFAR100(root='./data/CIFAR', train=True, download=True)
CIFAR100_trainset.transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomCrop(32, padding=4),ToTensor(),Normalize([0.5017, 0.4866, 0.4409],[0.2009, 0.1984, 0.2023]),])
CIFAR100_testset = datasets.CIFAR100(root='./data/CIFAR', train=False, download=True)
CIFAR100_testset.transform = transforms.Compose([ToTensor(), Normalize([0.5017, 0.4866, 0.4409],[0.2009, 0.1984, 0.2023]),])
CIFAR100_trainset = DataLoader(CIFAR100_trainset, batch_size=128, shuffle=True)
CIFAR100_testset = DataLoader(CIFAR100_testset, batch_size=128)

100%|██████████| 9912422/9912422 [00:00<00:00, 257931442.25it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 88098686.42it/s]


Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 77912368.58it/s]

...

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



100%|██████████| 170498071/170498071 [00:12<00:00, 13126558.22it/s]


Extracting ./data/CIFAR/cifar-10-python.tar.gz to ./data/CIFAR
Files already downloaded and verified


100%|██████████| 169001437/169001437 [00:12<00:00, 13030374.41it/s]


Extracting ./data/CIFAR/cifar-100-python.tar.gz to ./data/CIFAR
Files already downloaded and verified


# **Base Network - ResNet-18**

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_planes != self.expansion*planes:
            conv2d = nn.Conv2d(in_planes, self.expansion*planes,kernel_size=1, stride=stride, bias=False)
            bnorm2d = nn.BatchNorm2d(self.expansion*planes)
            self.shortcut = nn.Sequential(conv2d, bnorm2d)

    def forward(self, x):
        output = F.relu(self.bn1(self.conv1(x)))
        return F.relu(self.bn2(self.conv2(output)) + self.shortcut(x))


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_planes != self.expansion*planes:
            conv2d = nn.Conv2d(in_planes, self.expansion*planes,kernel_size=1, stride=stride, bias=False)
            bnorm2d = nn.BatchNorm2d(self.expansion*planes)
            self.shortcut = nn.Sequential(conv2d,bnorm2d)

    def forward(self, x):
        output = F.relu(self.bn1(self.conv1(x)))
        output = F.relu(self.bn2(self.conv2(output)))
        return F.relu(self.bn3(self.conv3(output)) + self.shortcut(x))


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.num_classes = num_classes
        self.linear = nn.Linear(512*block.expansion, self.num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = F.relu(self.bn1(self.conv1(x)))
        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)
        output = self.layer4(output)
        output = F.avg_pool2d(output, 4)
        output = output.view(output.size(0), -1)
        return self.linear(output)


def ResNet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)
     

# **Model Training**

In [ ]:
config = {
    'model_name' : 'resnet',
    'dataset_name' : 'CIFAR10',
    'batch_size' : 128,
    'lr': 1e-1,
    'weight_decay': 1e-4,
    'image_size' : 32,
    'epochs' : 20,
    'num_classes' : 10,
    'feature_extract' : False,
    'log_every': 50,
    'mixup_alpha' : 1,
    'augmentations': True,
    'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'model_type': 'Baseline'
}

device = config['device']


def train_model(model, trainloader, testloader, criterion, optimizer, scheduler, model_type, datasetname, num_epochs=25):
  print('model_type - ', model_type)

  start_time = time.time()
  val_acc_history = []

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  device = config['device']

  for epoch in range(num_epochs):

    print('Epoch {}/{}'.format(epoch+1, num_epochs), '\n----------')

    for phase in ['train', 'valid']:
      if phase == 'train':
        model.train()

        running_loss = 0.0
        running_corrects = 0

        for batch_idx,(inputs, labels) in tqdm(enumerate(trainloader)):
          optimizer.zero_grad()

          if model_type == 'MixUp':
            inputs = inputs.to(device)
            labels = labels.to(device) 
            inputs, labels, second_labels, lam = mixup(inputs, labels, config['mixup_alpha'])
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, labels, second_labels, lam)
            _, preds = torch.max(outputs, 1)
            
          elif model_type == 'CutMix':
            inputs = inputs.to(device)
            labels = labels.to(device) 
            inputs, labels, second_labels, lam = cutmix(inputs, labels, config['mixup_alpha'])
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, labels, second_labels, lam)
            _, preds = torch.max(outputs, 1)

          else:
            inputs = inputs.to(device)
            labels = labels.to(device) 
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
          
          loss.backward()
          optimizer.step()
          

          if batch_idx % config['log_every'] == 0: wandb.log({f'Train - loss': loss.item()})

          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(trainloader.dataset)
        epoch_acc = running_corrects.double() / len(trainloader.dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        wandb.log({'Train - acc': epoch_acc})
      
      
      else:
        model.eval()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in testloader:
          inputs = inputs.to(device)
          labels = labels.to(device)
          optimizer.zero_grad()
          with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
          epoch_loss = running_loss / len(testloader.dataset)
          epoch_acc = running_corrects.double() / len(testloader.dataset)
        
        wandb.log({'Valid - loss': epoch_loss})
        wandb.log({'Valid - acc': epoch_acc})
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model_wts = copy.deepcopy(model.state_dict())


        print('------------------------------')
        val_acc_history.append(epoch_acc)

    if scheduler: scheduler.step()



  time_diff = time.time() - start_time
  print('Training complete in {:.0f}m {:.0f}s'.format(time_diff // 60, time_diff % 60))
  print('Best validation Acc: {:4f}'.format(best_acc))
  model.load_state_dict(best_model_wts)
  return model, val_acc_history

def initialize_model(model_name, dataset_name, feature_extract=False, use_pretrained=False):
    input_size = 0

    if dataset_name == "mnist":
      input_size = 28
      num_classes = 10
    elif dataset_name =="cifar10":
      input_size = 32
      num_classes = 10
    else:
      input_size = 32
      num_classes = 100
    model_ft = ResNet18(num_classes)
    return model_ft, input_size

# **Training ResNet-18 on MNIST Dataset**

In [ ]:
config['model_name'] = 'resnet'
config['dataset_name'] = 'mnist'
model_ft, input_size = initialize_model(config['model_name'], config['dataset_name'])
optimizer_ft = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = 0.0005, nesterov = True)
scheduler_ft = CosineAnnealingLR(optimizer_ft, T_max = 60, eta_min = 1e-6 )
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(
      config=config,
      project='Project',
      group= 'baseline_mnist',
      save_code=True,
  ):
  model_ft, hist = train_model(
      model_ft, mnist_trainset,
      mnist_testset, criterion,
      optimizer_ft, scheduler_ft,
      num_epochs=config['epochs'],
      model_type= "Baseline",
      datasetname = config['dataset_name']
      )


wandb: Currently logged in as: esse-dawut (ddmaster). Use `wandb login --relogin` to force relogin


DAmethod -  Baseline
Epoch 1/20
----------


469it [01:21,  5.78it/s]


train Loss: 0.4012 Acc: 0.8984
valid Loss: 0.0837 Acc: 0.9734
Saving model...
------------------------------

Epoch 2/20
----------


469it [01:14,  6.31it/s]


...

469it [01:16,  6.14it/s]


train Loss: 0.0171 Acc: 0.9952
valid Loss: 0.0644 Acc: 0.9803
------------------------------

Epoch 20/20
----------


469it [01:13,  6.36it/s]


train Loss: 0.0186 Acc: 0.9946
valid Loss: 0.0301 Acc: 0.9913
------------------------------

Best model saved.
Training complete in 27m 10s
Best val Acc: 0.992200


Train - acc,▁▇██████████████████
Train - loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
Valid - acc,▂▁▆▇▅▆▆▅▇▅▇▄█▇▇▇█▇▄█
Valid - loss,▇█▃▂▄▂▃▄▂▄▂▆▁▂▂▃▁▂▅▁
Train - acc,0.99463
Train - loss,0.0023
Valid - acc,0.9913
Valid - loss,0.03008


# **Training ResNet-18 on CIFAR-10 Dataset**

In [ ]:
config['model_name'] = 'resnet'
config['dataset_name'] = 'cifar10'
model_ft, input_size = initialize_model(config['model_name'], config['dataset_name'])
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = 0.0005, nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6)
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(
        config=config,
        project='Project',
        group= 'baseline_cifar10',
        save_code=True,
    ):
  model_ft, hist = train_model(
      model_ft,
      CIFAR10_trainset,
      CIFAR10_testset,
      criterion,
      optimizer,
      scheduler,
      num_epochs=20,
      model_type = 'Baseline',
      datasetname = config['dataset_name']
      )


DAmethod -  Baseline
Epoch 1/20
----------


391it [01:09,  5.61it/s]


train Loss: 2.1439 Acc: 0.2594
valid Loss: 1.6627 Acc: 0.3824
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:08,  5.68it/s]


train Loss: 1.5354 Acc: 0.4316
valid Loss: 1.4112 Acc: 0.4866
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:10,  5.53it/s]


train Loss: 0.3728 Acc: 0.8718
valid Loss: 0.4966 Acc: 0.8354
Saving model...
------------------------------

Epoch 20/20
----------


391it [01:10,  5.56it/s]


train Loss: 0.3635 Acc: 0.8760
valid Loss: 0.5040 Acc: 0.8341
------------------------------

Best model saved.
Training complete in 24m 59s
Best val Acc: 0.835400


Train - acc,▁▃▄▅▆▆▇▇▇▇▇█████████
Train - loss,█▇▆▆▅▄▅▄▃▃▃▃▃▃▂▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▂▁▂
Valid - acc,▁▃▄▅▅▅▆▇▆▇▇▇█▇▇▇████
Valid - loss,█▆▆▄▄▄▃▃▃▂▂▂▁▃▂▂▁▁▁▁
Train - acc,0.87602
Train - loss,0.37879
Valid - acc,0.8341
Valid - loss,0.504


# **Training ResNet-18 on CIFAR-100 Dataset**

In [ ]:
config['dataset_name'] = 'cifar100'
config['model_name'] = 'resnet'

model_ft, input_size = initialize_model(config['model_name'], config['dataset_name'])
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = 0.0005, nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6)
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(
        config=config,
        project='Project',
        group= 'baseline_cifar100',
        save_code=True,
    ):
  model_ft, hist = train_model(
      model_ft,
      CIFAR100_trainset,
      CIFAR100_testset,
      criterion,
      optimizer,
      scheduler,
      num_epochs=config['epochs'],
      model_type = 'Baseline',
      datasetname = config['dataset_name'])

DAmethod -  Baseline
Epoch 1/20
----------


391it [01:10,  5.51it/s]


train Loss: 3.9111 Acc: 0.1001
valid Loss: 3.5916 Acc: 0.1392
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:09,  5.66it/s]


train Loss: 3.1986 Acc: 0.2076
valid Loss: 3.0796 Acc: 0.2376
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:09,  5.59it/s]


train Loss: 1.0433 Acc: 0.6929
valid Loss: 1.6284 Acc: 0.5704
------------------------------

Epoch 20/20
----------


391it [01:09,  5.65it/s]


train Loss: 1.0082 Acc: 0.7034
valid Loss: 1.5192 Acc: 0.5840
------------------------------

Best model saved.
Training complete in 24m 57s
Best val Acc: 0.606400


Train - acc,▁▂▃▄▅▆▆▆▇▇▇▇▇▇▇█████
Train - loss,█▇▆▅▅▄▄▃▄▃▃▄▃▃▂▃▂▃▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▂▁▂
Valid - acc,▁▂▄▅▅▆▆▇▇▆▇▇▇▇▇▇██▇█
Valid - loss,█▆▅▄▃▃▂▂▂▃▂▂▂▂▂▂▁▁▂▁
Train - acc,0.70342
Train - loss,1.06535
Valid - acc,0.584
Valid - loss,1.51922


# **Mixup**

In [ ]:
def mixup(inputs, labels, alpha):
  batch_size = inputs.size(0)
  shuffled_batch = torch.randperm(batch_size).to(config['device'])
  second_inputs = inputs[shuffled_batch]
  second_labels = labels[shuffled_batch]
  lam = np.random.beta(alpha, alpha)
  mixup_inputs = inputs * lam + second_inputs * (1 - lam)
  return mixup_inputs, labels, second_labels, lam

def mixup_criterion(criterion, output, original_labels, shuffled_labels, lam):
    return lam * criterion(output, original_labels) + (1 - lam) * criterion(output, shuffled_labels)


# **Training ResNet-18 + Mix Up on CIFAR-10 Dataset**

In [ ]:
config['dataset_name'] = 'CIFAR10'
config['model_name'] =  'resnet'
config['model_type'] = 'MixUp'
model_ft, input_size = initialize_model(config['model_name'], config['dataset_name'] )

optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = config['weight_decay'], nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6 )
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(
    config=config,
    project='Project',
    save_code=True,
    group='resnet18_cifar10_mixup'):
  
  model_ft, hist = train_model(
      model_ft,
      CIFAR10_trainset,
      CIFAR10_testset,
      criterion,
      optimizer,
      model_type=config['model_type'],
      datasetname=config['dataset_name'],
      num_epochs=config['epochs'],
      scheduler=scheduler
      )


DAmethod -  MixUp
Epoch 1/20
----------


391it [01:09,  5.61it/s]


train Loss: 2.2598 Acc: 0.1881
valid Loss: 1.7518 Acc: 0.3710
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:10,  5.56it/s]


train Loss: 1.9485 Acc: 0.2480
valid Loss: 1.5301 Acc: 0.4511
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:09,  5.62it/s]


train Loss: 1.1954 Acc: 0.4496
valid Loss: 0.6032 Acc: 0.8435
Saving model...
------------------------------

Epoch 20/20
----------


391it [01:09,  5.66it/s]


train Loss: 1.1710 Acc: 0.4958
valid Loss: 0.6086 Acc: 0.8405
------------------------------

Best model saved.
Training complete in 25m 3s
Best val Acc: 0.843500


Train - acc,▁▂▃▄▄▅▅▆▆▇▆▆▇▇▇▇█▇▇█
Train - loss,██▇▇▅▇▆▇▄▅▅▇▆▆▄▄▅▃▃▂▅▅▆▃▃▆▁▅▂▆▅▅▄▅▅▅▁▅▄▅
Valid - acc,▁▂▃▃▃▄▅▆▆▆▇▇▇▇██████
Valid - loss,█▇▆▆▆▄▄▃▃▃▂▂▁▂▁▁▁▁▁▁
Train - acc,0.49576
Train - loss,1.5986
Valid - acc,0.8405
Valid - loss,0.60859


# **Training ResNet-18 + Mix Up on CIFAR-100 Dataset**

In [ ]:
config['dataset_name'] = 'CIFAR100'
config['model_name'] =  'resnet'
config['model_type'] = 'MixUp'
model_ft, input_size = initialize_model(config['model_name'], config['dataset_name'] )
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = config['weight_decay'], nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6 )
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(
    config=config,
    project='Project',
    save_code=True,
    group='resnet18_cifar100_mixup'):
  
    model_ft, hist = train_model(
      model_ft,
      CIFAR100_trainset,
      CIFAR100_testset,
      criterion,
      optimizer,
      model_type=config['model_type'],
      datasetname=config['dataset_name'],
      num_epochs=config['epochs'],
      scheduler=scheduler
      )


     

DAmethod -  MixUp
Epoch 1/20
----------


391it [01:09,  5.64it/s]


train Loss: 4.2514 Acc: 0.0509
valid Loss: 3.7574 Acc: 0.1166
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:09,  5.62it/s]


train Loss: 3.8943 Acc: 0.0797
valid Loss: 3.2231 Acc: 0.2081
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:09,  5.61it/s]


train Loss: 2.3605 Acc: 0.3318
valid Loss: 1.4690 Acc: 0.6296
Saving model...
------------------------------

Epoch 20/20
----------


391it [01:10,  5.55it/s]


train Loss: 2.3192 Acc: 0.3451
valid Loss: 1.3766 Acc: 0.6472
Saving model...
------------------------------

Best model saved.
Training complete in 25m 1s
Best val Acc: 0.647200


Train - acc,▁▂▃▃▄▅▅▅▅▆▆▆▇▇▇▇████
Train - loss,█▇█▇▇▅▅▅▄▃▄▆▆▅▆▅▃▃▄▄▅▆▁▆▅▄▃▂▃▃▅▃▅▂▅▁▅▅▂▄
Valid - acc,▁▂▃▄▅▅▅▆▆▆▇▇▇▇▇█▇███
Valid - loss,█▆▆▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
Train - acc,0.3451
Train - loss,2.73077
Valid - acc,0.6472
Valid - loss,1.37655


# **Cut Mix**

In [ ]:
def cutmix(inputs, labels, alpha):
  batch_size = inputs.size(0)
  shuffled_batch = torch.randperm(batch_size).to(config['device'])
  second_inputs = inputs[shuffled_batch]
  second_labels = labels[shuffled_batch]
  lam = np.random.beta(alpha, alpha)
  
  mixup_inputs = torch.clone(inputs)
  W = mixup_inputs.size(2)
  H = mixup_inputs.size(3)

  for i in range(batch_size):
    height = int(W*np.sqrt(1-lam)) 
    width = int(H*np.sqrt(1-lam))
    randx = np.random.randint(0,W-width)
    randy = np.random.randint(0,H-height)

    mixup_inputs[i,:,randy:randy + height,randx:randx + width] = second_inputs[i,:,randy:randy + height,randx:randx + width]

  return mixup_inputs, labels, second_labels, lam

def mixup_criterion(criterion, output, original_labels, shuffled_labels, lam):
    return lam * criterion(output, original_labels) + (1 - lam) * criterion(output, shuffled_labels)

# **Training ResNet-18 + CutMix on CIFAR-10 Dataset**

In [ ]:
config['dataset_name'] = 'CIFAR10'
config['model_name'] =  'resnet'
config['model_type'] = 'CutMix'
model_ft, input_size = initialize_model('resnet', 'CIFAR10')
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = config['weight_decay'], nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6 )
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(
    config=config,
    project='Project',
    save_code=True,
    group='resnet18_cifar10_cutmix'):
  model_ft, hist = train_model(
      model_ft,
      CIFAR10_trainset,
      CIFAR10_testset,
      criterion,
      optimizer,
      model_type=config['model_type'],
      datasetname=config['dataset_name'],
      num_epochs=config['epochs'],
      scheduler=scheduler
      )

DAmethod -  CutMix
Epoch 1/20
----------


391it [01:12,  5.41it/s]


train Loss: 2.1714 Acc: 0.1787
valid Loss: 1.6815 Acc: 0.3771
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:12,  5.36it/s]


train Loss: 1.8696 Acc: 0.2451
valid Loss: 1.3753 Acc: 0.5042
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:12,  5.41it/s]


train Loss: 1.1281 Acc: 0.4045
valid Loss: 0.4870 Acc: 0.8686
Saving model...
------------------------------

Epoch 20/20
----------


391it [01:11,  5.47it/s]


train Loss: 1.1227 Acc: 0.3951
valid Loss: 0.5426 Acc: 0.8453
------------------------------

Best model saved.
Training complete in 25m 44s
Best val Acc: 0.868600


Train - acc,▁▃▄▅▅▆▆▇▇▇▇▇▇▇▇████▇
Train - loss,█▇▆▆▆▅▅▆▆▆▅▆▅▆▃▃▄▄▃▂▅▄▂▅▅▃▁▄▅▂▃▅▄▃▂▃▅▅▃▅
Valid - acc,▁▃▃▄▄▅▆▆▆▆▇▇▇▇█▇████
Valid - loss,█▆▆▅▅▄▃▂▃▃▂▂▂▂▁▂▁▁▁▁
Train - acc,0.39508
Train - loss,1.51644
Valid - acc,0.8453
Valid - loss,0.54261


# **Training ResNet-18 + CutMix on CIFAR-100 Dataset**

In [ ]:
config['dataset_name'] = 'CIFAR100'
config['model_name'] =  'resnet'
config['model_type'] = 'CutMix'
model_ft, input_size = initialize_model('resnet', 'CIFAR100')
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = config['weight_decay'], nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6 )
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(config=config, project='Project', save_code=True, group='resnet18_cifar100_cutmix'):
  model_ft, hist = train_model(
      model_ft,
      CIFAR100_trainset,
      CIFAR100_testset,
      criterion,
      optimizer,
      model_type=config['model_type'],
      datasetname=config['dataset_name'],
      num_epochs=config['epochs'],
      scheduler=scheduler
      )

DAmethod -  CutMix
Epoch 1/20
----------


391it [01:11,  5.44it/s]


train Loss: 4.2562 Acc: 0.0379
valid Loss: 3.6775 Acc: 0.1310
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:12,  5.42it/s]


train Loss: 3.9412 Acc: 0.0669
valid Loss: 3.4524 Acc: 0.1708
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:12,  5.40it/s]


train Loss: 2.3556 Acc: 0.2482
valid Loss: 1.3236 Acc: 0.6384
Saving model...
------------------------------

Epoch 20/20
----------


391it [01:11,  5.47it/s]


train Loss: 2.3588 Acc: 0.2718
valid Loss: 1.3645 Acc: 0.6429
Saving model...
------------------------------

Best model saved.
Training complete in 25m 43s
Best val Acc: 0.642900


Train - acc,▁▂▃▃▄▄▅▆▅▅▆▆▇▇▇███▇█
Train - loss,█▇▇▇▇▅▇▅▆▅▆▅▃▃▄▅▅▅▅▄▅▅▁▅▅▅▅▃▅▃▃▃▃▄▃▁▃▃▄▁
Valid - acc,▁▂▃▄▄▅▅▆▆▆▇▇▇▇██████
Valid - loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
Train - acc,0.27182
Train - loss,1.41847
Valid - acc,0.6429
Valid - loss,1.36455


# **AugMix**

In [ ]:
def int_parameter(level,maxval):
  return int(level*maxval/10)

def float_parameter(level,maxval):
  return float(level)*maxval/10.

def sample_level(n):
  return np.random.uniform(low=0.1,high=n)

def autocontrast(pil_img, _ , image_size):
  return ImageOps.autocontrast(pil_img)

def equalize(pil_img, _, image_size):
  return ImageOps.equalize(pil_img)

def posterize(pil_img, level, image_size):
  return ImageOps.posterize(pil_img, 4 - int_parameter(sample_level(level), 4))

def rotate(pil_img, level, image_size):
  degrees = int_parameter(sample_level(level), 30)
  if np.random.uniform() > 0.5:
    degrees = -degrees
  return pil_img.rotate(degrees, resample=Image.BILINEAR)

def solarize(pil_img, level, image_size):
  return ImageOps.solarize(pil_img, 256 - int_parameter(sample_level(level), 256))

def shear_x(pil_img, level, image_size):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((image_size, image_size),Image.AFFINE, (1, level, 0, 0, 1, 0),resample=Image.BILINEAR)

def shear_y(pil_img, level, image_size):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((image_size, image_size),Image.AFFINE, (1, 0, 0, level, 1, 0),resample=Image.BILINEAR)

def translate_x(pil_img, level, image_size):
  level = int_parameter(sample_level(level), image_size / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((image_size, image_size), Image.AFFINE, (1, 0, level, 0, 1, 0), resample=Image.BILINEAR)

def translate_y(pil_img, level, image_size):
  level = int_parameter(sample_level(level), image_size / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((image_size, image_size), Image.AFFINE, (1, 0, 0, 0, 1, level), resample=Image.BILINEAR)

def color(pil_img, level, image_size):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Color(pil_img).enhance(level)

def contrast(pil_img, level, image_size):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Contrast(pil_img).enhance(level)

def brightness(pil_img, level, image_size):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Brightness(pil_img).enhance(level)

def sharpness(pil_img, level, image_size):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Sharpness(pil_img).enhance(level)

augmentations = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y
]

augmentations_all = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y, color, contrast, brightness, sharpness
]

def augmix(image, preprocess, dataset_name, mixture_width:int=3, aug_severity = 3, all_ops = True):
  
  if dataset_name == "mnist":
    size = 28
  else:
    size = 32

  aug_list = augmentations
  if all_ops:
    aug_list = augmentations_all
  
  ws = np.float32(np.random.dirichlet([1] * mixture_width))
  m = np.float32(np.random.beta(1,1))

  mix = torch.zeros_like(preprocess(image))

  for i in range(mixture_width):
    image_aug = image.copy()
    depth = np.random.randint(1, 4)
    for _ in range(depth):
      op = np.random.choice(aug_list)
      image_aug = op(image_aug, aug_severity, size)
    mix += ws[i] * preprocess(image_aug)
  
  
  mixed = (1-m) * preprocess(image) + m * mix

  return mixed

class AugMixDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,preprocess,dataset_name,all_ops=True,jsd_loss=False):
    self.dataset = dataset
    self.preprocess = preprocess
    self.jsd_loss = jsd_loss
    self.dataset_name = dataset_name
    self.all_ops = all_ops
  
  def __getitem__(self, i):
    x, y = self.dataset[i]
    if self.jsd_loss:
      im_tuple = (
          self.preprocess(x), 
          augmix(x, self.preprocess, self.dataset_name, self.all_ops),
          augmix(x, self.preprocess, self.dataset_name, self.all_ops),
          )
      return im_tuple,y
    else:
      return augmix(x, self.preprocess, self.dataset_name, self.all_ops), y
  
  def __len__(self):
    return len(self.dataset)


# **Training ResNet-18 + AugMix on CIFAR-10 Dataset**

In [ ]:
config['dataset_name'] = 'CIFAR10'
config['model_name'] =  'resnet'
config['model_type'] = 'AugMix'
model_ft, input_size = initialize_model('resnet', 'CIFAR10')
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = config['weight_decay'], nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6 )
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(config=config, project='Project', save_code=True, group='resnet18_cifar10_augmix'):
  model_ft, hist = train_model(
      model_ft,
      CIFAR10_trainset,
      CIFAR10_testset,
      criterion,
      optimizer,
      model_type=config['model_type'],
      datasetname=config['dataset_name'],
      num_epochs=config['epochs'],
      scheduler=scheduler
      )


DAmethod -  AugMix
Epoch 1/20
----------


391it [01:09,  5.61it/s]


train Loss: 1.9922 Acc: 0.3119
valid Loss: 1.5976 Acc: 0.4167
Saving model...
------------------------------

Epoch 2/20
----------


391it [01:08,  5.68it/s]


train Loss: 1.3712 Acc: 0.4996
valid Loss: 1.2276 Acc: 0.5600
Saving model...
------------------------------

Epoch 3/20
----------


...

391it [01:09,  5.61it/s]


train Loss: 0.2394 Acc: 0.9170
valid Loss: 0.3792 Acc: 0.8767
Saving model...
------------------------------

Epoch 20/20
----------


391it [01:08,  5.68it/s]


train Loss: 0.2205 Acc: 0.9226
valid Loss: 0.3759 Acc: 0.8800
Saving model...
------------------------------

Best model saved.
Training complete in 24m 50s
Best val Acc: 0.880000


Train - acc,▁▃▄▅▆▆▇▇▇▇▇▇▇███████
Train - loss,█▆▆▆▅▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▂▁▂▁▁▁▁▁▁
Valid - acc,▁▃▅▅▆▆▆▇▇▇▇▇▇▇██████
Valid - loss,█▆▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
Train - acc,0.9226
Train - loss,0.25736
Valid - acc,0.88
Valid - loss,0.37591


# **Training ResNet-18 + AugMix on CIFAR-100 Dataset**

In [ ]:
config['dataset_name'] = 'CIFAR100'
config['model_name'] =  'resnet'
config['model_type'] = 'AugMix'
model_ft, input_size = initialize_model(config['model_name'], config['dataset_name'] )
optimizer = optim.SGD(model_ft.parameters(), lr=config['lr'], momentum=0.9, weight_decay = config['weight_decay'], nesterov = True)
scheduler = CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-6)
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to(device)

with wandb.init(config=config, project='Project', save_code=True, group='resnet18_cifar100_augmix'):
  model_ft, hist = train_model(
      model_ft,
      CIFAR100_trainset,
      CIFAR100_testset,
      criterion,
      optimizer,
      model_type=config['model_type'],
      datasetname=config['dataset_name'],
      num_epochs=config['epochs'],
      scheduler=scheduler
      )